# Wildfire Hazard: Data Retrieval/Prep 2 - Vector Layers (roads, streams, etc)

Program: wildfire_prep2.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook retrieves and prepares data for wildfire hazard analysis  
Date: May 9, 2025

In [ ]:
import arcpy
import os
import requests
import zipfile

# Get project directory
aprx = arcpy.mp.ArcGISProject("CURRENT")
gis_dir = os.path.dirname(aprx.filePath)
project_dir = os.path.dirname(gis_dir)

# Data Directory structure
raw_dir = os.path.join(project_dir, "data", "raw")
extract_dir = os.path.join(project_dir, "data", "extracted")
proc_dir = os.path.join(project_dir, "data", "processed")

# Create folders if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(extract_dir, exist_ok=True)
os.makedirs(proc_dir, exist_ok=True)

# Set ArcPy Workspace & overwrite options
arcpy.env.workspace = raw_dir
arcpy.env.overwriteOutput = True

# Define download url input and filename output (for roads, hydro, utilities, and buildings)
urls = [
    ("https://nsgi.novascotia.ca/WSF_DDS/DDS.svc/DownloadFile?tkey=kNNpTdP4QuNRSYtt&id=113129", "roads.zip"),
    ("https://nsgi.novascotia.ca/WSF_DDS/DDS.svc/DownloadFile?tkey=kNNpTdP4QuNRSYtt&id=38905", "hydro.zip"),
    ("https://nsgi.novascotia.ca/WSF_DDS/DDS.svc/DownloadFile?tkey=kNNpTdP4QuNRSYtt&id=36775", "utilities.zip"),
    ("https://nsgi.novascotia.ca/WSF_DDS/DDS.svc/DownloadFile?tkey=kNNpTdP4QuNRSYtt&id=36733", "buildings.zip")
]

# Download and extract the files
def dl_file(url, path):
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)
        
        # Check if request was successful
        if response.status_code == 200:
            with open(path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192): 
                    file.write(chunk)
            print(f"Downloaded: {path}")
        else:
            print(f"Failed to download {url}, status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

def extract_zip(zip_path, extract_folder):
    os.makedirs(extract_folder, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        print(f"Extracted to {extract_folder}")

# Loop through URLs and process
for url, filename in urls:
    file_path = os.path.join(raw_dir, filename)
    dl_file(url, file_path)
    extract_folder = os.path.join(extract_dir, os.path.splitext(filename)[0])
    extract_zip(file_path, extract_folder)